# Using LLMs in Humanities Research via API

## Session 2 14.00-15.30 - Working with LLMs via API

Through practical examples, we will explore prompt engineering techniques for tasks such as concept mining and named entity recognition in textual data.

## Session Outline

- **Prompt Engineering**: Techniques for crafting effective prompts to guide LLMs in generating relevant and accurate responses.
- **Concept Mining**: Using LLMs to extract key concepts from text, enabling researchers to identify important themes and ideas.
- **Named Entity Recognition (NER)**: Implementing NER to identify and classify entities in text, such as people, organizations, and locations.



## BSSDH 2025 Workshop Data

Before we start exploring the API, let's take a look at the corpus of documents we will be working. 
Data for workshops in [Baltic Summer School of Digital Humanities 2025](https://www.digitalhumanities.lv/bssdh/2025/about/)

**Repository:** https://github.com/LNB-DH/BSSDH_2025_workshop_data




## CORPUS OVERVIEW


1. SOURCE MATERIAL
------------------

| Periodical | Details |
|------------|---------|
| "Rigasche Zeitung" (RZei) (1918–1919) | - **Data file:** `Rigasche_Zeitung_1918_1919.zip`<br>- **Download Rigasche Zeitung:** https://github.com/LNB-DH/BSSDH_2025_workshop_data/raw/main/data/Rigasche_Zeitung_1918_1919.zip<br>- Morning newspaper, intermittently published from 1778 to 1919 in Riga.<br>- Language: German (Fraktur script)<br>- Once the most popular morning paper in the Baltic provinces of the Russian Empire.<br>- Covered general political and economic news in Riga, the Baltics, the Russian Empire, and internationally.<br>- Historical context: World War I, Latvian War of Independence.<br>- Link: https://periodika.lv/#periodicalMeta:234;-1<br>- More info: https://enciklopedija.lv/skirklis/163962 |
| "Latvian Economic Review" (LERQ) (1936–1940) | - **Data file:** `Latvian_Economic_Review_1936_1940.zip`<br>- **Download Latvian Economic Review:** https://github.com/LNB-DH/BSSDH_2025_workshop_data/raw/main/data/Latvian_Economic_Review_1936_1940.zip<br>- Full title: "Latvian Economic Review: A quarterly review of trade, industry and agriculture".<br>- Language: English (modern)<br>- Published by the Latvian Chamber of Commerce and Industry (established 1934).<br>- Focused on cross-border representation of Latvian economy during the Great Depression, increasing state control, push for autarky, and start of WWII.<br>- Link: https://periodika.lv/#periodicalItem:620 |

2. CORPUS INFORMATION
----------------------

| Metric | RZei | LERQ |
|--------|------|------|
| Token Count (words) | 5.37 million | 0.5 million |
| Issue Count | 359 issues | 18 issues |
| Segment (Article <=> File) Count | 4,597 | 419 |
| Language | German | English |
| Script | Fraktur | Modern |

Filename Structure:
-------------------
Format: [periodical][year][volume#*][issue#]_[page#]_[[plaintext]]_[segment#]

Example: `lerq1936s01n02_031_plaintext_s17.txt`
         → 17th segment from LERQ, Issue 2, 1936, page 31.

*Volume value in corpus is one in all cases.

3. METHODOLOGY
---------------

| Step | Description |
|------|-------------|
| 3.1. Source Access | Digitised issues obtained from the National Library of Latvia (https://periodika.lv/) |
| 3.2. Processing & OCR | CCS docWORKS & ABBYY FineReader 9.0<br>- LERQ has better OCR quality than RZei<br>- No further data cleaning/normalization |
| 3.3. Metadata Added | Fields: title, author, uri<br>- Author info available in:<br>&nbsp;&nbsp;&nbsp;&nbsp;LERQ: 4 cases (0.95%)<br>&nbsp;&nbsp;&nbsp;&nbsp;RZei: 325 cases (7.05%)<br>- Title availability:<br>&nbsp;&nbsp;&nbsp;&nbsp;LERQ: 95.7%, RZei: 99.15%<br>- URI coverage: 100% for both<br>- URIs point to LNB DOM system |


## Extracting documents

We could extract documents manually by downloading the appropriate zip file and extracting files by *hand* using file extracting capabilities built into your Operating System(Windows has built in extractor) or using external program such as 7-zip, WinRAR, etc. However, it is more replicable and convenient to use a script that will do this for us. We would supply a url or file name and the script would download the file, extract it to approparite location, and return a list of files that were extracted.

### Additional considerations when extracting documents

* Where will be extracted files be stored? - Ideally we would have a same relative structure when extracting files locally and on remote server such as Google Colab.
* How will we handle file names? - Usually we would like to keep the original file names, but we might want to add some additional information such as source or date of extraction.
* How will we handle errors? - We should consider what to do if the file cannot be downloaded or extracted. Should we skip it or raise an error?

### Extracting Latvian Economic Review

For this session we will extract Latvian Economic Review (LERQ) corpus. We will use a script that will download the file, extract it to appropriate location, and return a list of files that were extracted.

We will write a function in Python that will do this for us. The function will take a URL or file name as an argument and will download file from url and then extract it. We will have a default location where the files will be extracted, but we can also specify a different location if needed.

```python



In [4]:
url = "https://github.com/LNB-DH/BSSDH_2025_workshop_data/raw/main/data/Latvian_Economic_Review_1936_1940.zip"
print("Will extract data from", url)
# next we define a function that will download and extract the zip file, this way we can reuse it later if needed
# we can also set some default values for the arguments, so we do not have to specify
# default values always come after the mandatory arguments in Python functions
def extract_zip(url, output_dir="data", verbose=False):
    # we could have imported these at the top, but we want to keep the script self-contained
    import requests  # this should be cached by notebooks, so it **should** not require importing it every time
    from zipfile import ZipFile
    from io import BytesIO

    # In verbose mode let's some extra information about the download and extraction process
    # This is useful for debugging and understanding the flow of the script
    from datetime import datetime
    if verbose:
        download_start = datetime.now()
        # we print start time including milliseconds
        print(f"Starting download at {download_start.strftime('%Y-%m-%d %H:%M:%S.%f')}")
        print("Starting download from", url)
    response = requests.get(url)
    if verbose:
        download_finish = datetime.now()
        print(f"Download finished at {download_finish.strftime('%Y-%m-%d %H:%M:%S.%f')} taking {download_finish - download_start} seconds")
    if response.status_code == 200: # it is possible a request fails, e.g. if the URL is incorrect
        if verbose:
            extract_start = datetime.now()
            print(f"Starting extraction to {output_dir} at {extract_start.strftime('%Y-%m-%d %H:%M:%S.%f')}")
        with ZipFile(BytesIO(response.content)) as zf:
            zf.extractall(output_dir)
        if verbose:
            extract_end = datetime.now()
            print(f"Extraction finished at {extract_end.strftime('%Y-%m-%d %H:%M:%S.%f')} taking {extract_end - extract_start} seconds")
            print(f"Total time taken: {extract_end - download_start} seconds")
    else:
        print("Failed to download data:", response.status_code)

# now that we have our function defined, we can call it immediately
# note we do not supply all arguments, first one is mandatory, the rest are optional
# so we skip over output_dir in this case
extract_zip(url, verbose=True)

Will extract data from https://github.com/LNB-DH/BSSDH_2025_workshop_data/raw/main/data/Latvian_Economic_Review_1936_1940.zip
Starting download at 2025-07-31 11:40:24.272770
Starting download from https://github.com/LNB-DH/BSSDH_2025_workshop_data/raw/main/data/Latvian_Economic_Review_1936_1940.zip
Download finished at 2025-07-31 11:40:25.271504 taking 0:00:00.998734 seconds
Starting extraction to data at 2025-07-31 11:40:25.271504
Extraction finished at 2025-07-31 11:40:27.656727 taking 0:00:02.385223 seconds
Total time taken: 0:00:03.383957 seconds


### Getting information about extracted files

It is a good practice to double check what files were extracted and where they are located. We can do this by listing the files in the directory where we extracted them. We can use Python pathlib to do this. 
The goal is to double check that what we extacted matches what we expected. We can also check the file names and their structure to make sure they are correct.

```python

In [6]:
from pathlib import Path
extract_dir = Path("data") # note this is a a relative path, relative to the current working directory for the notebook
# let's check if the directory exists and how many files it contains
if extract_dir.exists():
    files = list(extract_dir.glob("*"))  # this will list all files in the directory
    print(f"Extracted {len(files)} files to {extract_dir}")
    for file in files:
        print(file.name)  # print the name of each file 
else:
    print(f"Directory {extract_dir} does not exist. Please check the extraction process.")

Extracted 1 files to data
Latvian_Economic_Review


### Getting information about subfolders in the extracted directory
Looks like we only have a single file but it is actually not a file but a directory. This is because we extracted a zip file that contains files under a single directory. 

Next we want to check how many total files we have and also how many files we have with *.txt extension. This will help us to understand how many files we can work with and if there are any files that we might want to exclude from our analysis.


In [ ]:
# let's check how many files total we have and how many files with *.txt extension counting all subfolders 
# this means we will perform a recursive search for all files in the directory

def analyze_directory_contents(directory_path, verbose=True):
    """
    Analyze the contents of a directory recursively and provide detailed information.
    
    Args:
        directory_path: Path object or string path to the directory
        verbose: If True, print detailed information about file types and structure
    
    Returns:
        dict: Dictionary containing analysis results
    """
    from pathlib import Path
    
    directory = Path(directory_path)
    
    if not directory.exists():
        print(f"Directory {directory} does not exist.")
        return None
    
    # Get all files recursively
    all_files = list(directory.rglob("*"))
    
    # Separate files from directories
    files_only = [f for f in all_files if f.is_file()]
    directories_only = [f for f in all_files if f.is_dir()]
    
    # Count files by extension
    file_extensions = {}
    for file in files_only:
        ext = file.suffix.lower()
        if ext == '':
            ext = '(no extension)'
        file_extensions[ext] = file_extensions.get(ext, 0) + 1
    
    # Count .txt files specifically
    txt_files = [f for f in files_only if f.suffix.lower() == '.txt']
    
    # Analysis results
    results = {
        'total_items': len(all_files),
        'total_files': len(files_only),
        'total_directories': len(directories_only),
        'txt_files': len(txt_files),
        'file_extensions': file_extensions,
        'txt_file_paths': txt_files
    }
    
    if verbose:
        print(f"📁 Directory Analysis: {directory}")
        print("=" * 50)
        print(f"Total items found (files + directories): {results['total_items']}")
        print(f"Total files: {results['total_files']}")
        print(f"Total directories: {results['total_directories']}")
        print(f"Text files (.txt): {results['txt_files']}")
        print()
        
        print("📊 File Extensions Summary:")
        print("-" * 30)
        for ext, count in sorted(file_extensions.items(), key=lambda x: x[1], reverse=True):
            print(f"  {ext}: {count} files")
        print()
        
        if len(directories_only) > 0:
            print("📂 Directory Structure:")
            print("-" * 30)
            for directory in sorted(directories_only):
                # Show relative path from the base directory
                relative_path = directory.relative_to(directory_path)
                print(f"  📁 {relative_path}")
            print()
        
        if len(txt_files) > 0 and len(txt_files) <= 10:
            print("📄 Sample .txt files:")
            print("-" * 30)
            for txt_file in sorted(txt_files)[:10]:
                relative_path = txt_file.relative_to(directory_path)
                file_size = txt_file.stat().st_size
                print(f"  📄 {relative_path} ({file_size:,} bytes)")
        elif len(txt_files) > 10:
            print(f"📄 First 10 .txt files (out of {len(txt_files)} total):")
            print("-" * 30)
            for txt_file in sorted(txt_files)[:10]:
                relative_path = txt_file.relative_to(directory_path)
                file_size = txt_file.stat().st_size
                print(f"  📄 {relative_path} ({file_size:,} bytes)")
            print(f"  ... and {len(txt_files) - 10} more .txt files")
    
    return results

# Now let's analyze our extracted directory
print("Analyzing extracted data directory...")
analysis_results = analyze_directory_contents(extract_dir, verbose=True)